In [2]:
%load_ext autoreload
%autoreload 2

import Structures
import EmotionExtractor


em = EmotionExtractor.EmotionExtractor('baseline.npy', 'baseline_mean_sd.pickle')





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Structures.py:105: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=[/input_2])`
  model = Model(input=[encoding_input], output=output)


In [3]:
import os                                                                                                             
from pydub import AudioSegment
import json
import pandas as pd
import pydub.exceptions

def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).next()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files: 
                if("qna_f" in file):
                    r.append(subdir + "\\" + file)
                    #print subdir + "\\" + file
    return r

#list_my_files= list_files("D:\\DataElisa\\Nokia")

from natsort import natsorted, ns
#mykeys= dict_files.keys()

def list_files_dir(dir):                                                                                                  
    r = {}                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:
        
        files = os.walk(subdir).next()[2]                                                                             
        if (len(files) > 0):
            r[subdir]= []
            for file in files: 
                    r.get(subdir).append(subdir + "\\" + file)
                    #print subdir + "\\" + file
            r[subdir] = natsorted(r.get(subdir), key=lambda y: y.lower())

    return r


dict_files = list_files_dir('D://DataElisa/PitchesAudio/')

#print(dict_files)





In [ ]:
#getting them 3 by 3

import traceback

dirnames= dict_files.keys()
dictionary_for_jason = {}

for name in dirnames:
    dictionary_for_jason[name] = []
    filenames=dict_files.get(name)
    for j in range(3,len(filenames)+3,3):
        #try:
        totfilenames =filenames[j-3:j]
        song = None
        for segment in totfilenames:
            if song==None:
                song= AudioSegment.from_mp3(segment)
            else:
                song.append(AudioSegment.from_mp3(segment))
            #9 secs
        result =  em.split_single_song(song)
        print result
        dictionary_for_jason[name].append(result)
            
        #except:
        #    traceback.print_exc()
        #    print("Not Decodable")
        #    return


WARNING (theano.gof.compilelock): Overriding existing lock by dead process '22932' (I am process '8736')


          Anger   Disgust      Fear  Happiness   Neutral   Sadness  Surprise
0  4.291657e-13  0.000034  0.952953   0.000323  0.026865  0.003158  0.016667
          Anger   Disgust     Fear  Happiness   Neutral  Sadness  Surprise
0  8.063478e-10  0.017833  0.97173   0.000123  0.006114  0.00003  0.004171
          Anger   Disgust      Fear  Happiness   Neutral   Sadness  Surprise
0  1.491547e-07  0.011666  0.299144   0.005737  0.633899  0.023656  0.025899
          Anger   Disgust      Fear  Happiness   Neutral   Sadness  Surprise
0  2.558783e-11  0.070679  0.000017   0.025359  0.902963  0.000861  0.000122
          Anger  Disgust     Fear  Happiness   Neutral   Sadness  Surprise
0  2.960038e-13  0.00379  0.00039    0.02347  0.968756  0.002871  0.000723
          Anger   Disgust      Fear  Happiness   Neutral  Sadness  Surprise
0  8.925677e-14  0.003386  0.000004     0.0682  0.866835  0.06157  0.000005
          Anger   Disgust      Fear  Happiness   Neutral   Sadness  Surprise
0  1.0021

In [8]:



result_all = pd.DataFrame()
to_json = []
count=0
for i in list_my_files:
    try:
        song = AudioSegment.from_mp3(i)
        result =  em.split_single_song(song)
        result["filename"] = i
    
        dict_to_append = result.to_dict('record')
    
    
    
        to_json.append(dict_to_append[0])
        count += 1
        print count
        if count % 1000 ==0:
            with open('jsn_nokia.txt', 'w') as outfile:
                json.dump(to_json, outfile)
                outfile.flush()
                outfile.close()
    except:
        print("Not Decodable")
    #print result.to_dict('list')
    #print json.dumps(to_json)
    #print result_all.to_json()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
Not Decodable
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
Not Decodable
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
Not Decodable
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
26

In [9]:
with open('jsn_nokia.txt', 'w') as outfile:
                json.dump(to_json, outfile)
                outfile.flush()
                outfile.close()

In [6]:
list_my_files

['D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000001.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000002.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000003.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000004.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000005.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000006.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000007.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000008.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000009.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000010.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000011.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000012.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000013.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000014.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000015.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\s

In [16]:
def list_files2(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).next()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files: 
                 if(".wav" in file):
                    r.append(subdir + "\\" + file)
                    #print subdir + "\\" + file
    return r


list_my_files2= list_files2("C:\\Users\\Stefano\\workspace\\EmodashSamples\\SamplesStandard\\AudioData\\AudioData")

for i in list_my_files2:
    print i
    if "h" in i:
        song = AudioSegment.from_mp3(i)
        print em.split_single_song(song)


C:\Users\Stefano\workspace\EmodashSamples\SamplesStandard\AudioData\AudioData\DC\a01.wav
          Anger   Disgust      Fear  Happiness   Neutral   Sadness  Surprise
0  7.595251e-09  0.004061  0.557963   0.005373  0.406514  0.004527  0.021562
C:\Users\Stefano\workspace\EmodashSamples\SamplesStandard\AudioData\AudioData\DC\a02.wav
          Anger       Disgust          Fear  Happiness  Neutral   Sadness  \
0  4.381788e-13  2.779374e-10  1.020624e-12   0.000093  0.01666  0.983244   

   Surprise  
0  0.000004  
C:\Users\Stefano\workspace\EmodashSamples\SamplesStandard\AudioData\AudioData\DC\a03.wav
          Anger  Disgust      Fear  Happiness   Neutral   Sadness  Surprise
0  6.231696e-13  0.00671  0.166298   0.006312  0.700172  0.012488  0.108021
C:\Users\Stefano\workspace\EmodashSamples\SamplesStandard\AudioData\AudioData\DC\a04.wav
          Anger       Disgust      Fear  Happiness   Neutral   Sadness  \
0  7.869479e-12  9.593674e-07  0.986515   0.000046  0.002871  0.010523   

   Sur

In [15]:
import pickle

with open('baseline_mean_sd.pickle', 'rb') as f:
        data_dict = pickle.load(f)


In [16]:
data_dict

{'mean': F1      0.216773
 F2      0.013962
 F3      2.873969
 F4      0.312087
 F5      0.232803
 F6      1.673590
 F7      0.008511
 F8      0.380520
 F9    -28.298735
 F10     1.551537
 F11    -0.410844
 F12    -0.310689
 F13    -0.317822
 F14     0.020887
 F15    -0.108504
 F16    -0.058962
 F17    -0.050438
 F18    -0.079914
 F19    -0.059984
 F20    -0.059330
 F21    -0.061714
 F22     0.017673
 F23     0.008670
 F24     0.029054
 F25     0.015675
 F26     0.015916
 F27     0.013855
 F28     0.017712
 F29     0.005443
 F30     0.010193
 F31     0.013305
 F32     0.016093
 F33     0.009804
 F34     0.020049
 dtype: float64, 'sd': F1     0.052421
 F2     0.010019
 F3     0.120432
 F4     0.047327
 F5     0.022069
 F6     0.349290
 F7     0.002751
 F8     0.111443
 F9     2.748299
 F10    0.560494
 F11    0.386987
 F12    0.235608
 F13    0.225211
 F14    0.161537
 F15    0.130334
 F16    0.104909
 F17    0.094200
 F18    0.114524
 F19    0.096467
 F20    0.108135
 F21    0.109757
 